<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Anomaly Detection in Credit Card
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style='font-size:20px;font-family:Arial;'><b>Credit Card Fraud Detection using K-Means Clustering:</b></p> 
<p style='font-size:16px;font-family:Arial;'>
Detecting fraudulent transactions is crucial for financial security. This approach leverages <b>K-Means clustering</b> to group similar transactions and identifies anomalies based on <b>Euclidean distance</b>, where fraud-like patterns deviate significantly from normal spending behaviors.
</p>

<ul style='font-size:16px;font-family:Arial;'>
    <li><strong>Anomaly Detection:</strong> Identifies outliers based on their distance from the cluster center, marking transactions that deviate from normal spending patterns.</li>
    <li><strong>Vector Embeddings:</strong> Converts categorical transaction data into vector representations to improve clustering accuracy.</li>
    <li><strong>Feature Engineering:</strong> Includes transaction amount, location, time, and merchant category to enhance fraud detection.</li>
    <li><strong>Dimensionality Reduction:</strong> Uses t-SNE to visualize clusters and detect fraudulent transactions that do not fit normal behavior.</li>
    <li><strong>Scalability:</strong> Works efficiently on large datasets by leveraging K-Means for unsupervised learning and anomaly detection.</li>
</ul>


<p style = 'font-size:18px;font-family:Arial;'><b>Why Vantage?</b></p>

<p style = 'font-size:16px;font-family:Arial;'>
    Teradata’s integration with <b>LLMs and hosting capabilities in-DB</b>, along with the Open Analytics Framework, would enable customers to run NLP models at scale. The key challenges noted for on-prem customers—such as data movement latency and lack of access to cloud models—are valid. By bringing language models within Vantage, Teradata can provide a significant advantage to on-prem customers by allowing them to run NLP models without needing to move large amounts of data to and from external services.
</p>

<hr style="height:1px;border:none;background-;">
<p style = 'font-size:18px;font-family:Arial;'><b>Downloading and installing additional software needed</b>

In [ ]:
# %%capture

!pip install wordcloud nltk --quiet

In [ ]:
# %%capture

!pip install --force-reinstall pillow --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>The above libraries have to be installed. Restart the kernel after executing these cells to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing <b> 0 0</b></i> (zero zero) and pressing <i>Enter</i>.</p>
</div>
<p style = 'font-size:16px;font-family:Arial;'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Standard libraries
import time
import warnings
import random

# Data manipluation and Visualization libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# Teradata libraries
from teradataml import *
from teradataml import (
    create_context, 
    delete_byom, 
    display,
    execute_sql,
    save_byom,
    remove_context,
    in_schema,
    KMeans,
    DataFrame,
    db_drop_table,
    db_drop_view
)
display.max_rows = 5

# NLP libraries
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)


# machine learning libraries
from sklearn.manifold import TSNE

# Suppress warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>1. Initiate a connection to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Anomaly_Detection_Credit_Card.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:18px;font-family:Arial;'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_CreditCard_cloud');" 
# takes about 20seconds, estimated space: 0 MB
%run -i ../run_procedure.py "call get_data('DEMO_CreditCard_local');" 
# takes about 35 seconds, estimated space: 11 MB

<p style = 'font-size:16px;font-family:Arial;'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>2. Generate real time data</b>

In [ ]:
tdf_cc = DataFrame(in_schema("DEMO_CreditCard","Credit_Card"))
tdf_cc

In [ ]:
tdf_cc.shape

In [ ]:
#Storing in database
copy_to_sql(df = tdf_cc, table_name = "credit_card_db", if_exists="replace")

In [ ]:
tdf = DataFrame.from_table('credit_card_db')
tdf.shape

<hr style="height:1px;border:none;background-;">
<b style = 'font-size:18px;font-family:Arial;'>2.1 Real-time Data Collection
</b>

<p style = 'font-size:16px;font-family:Arial;'>This simulated data mimics the real-time process, where transaction details are captured continuously, enabling anomaly detection in real-time credit card activity.</p>

In [ ]:
# Real time data collection
def fetching_real_time_data():
    contract_types = ["Cash loans", "Revolving loans"]
    genders = ["M", "F"]
    own_car = ["Y", "N"]
    family_status = ["Married", "Single", "Separated"]
    house_types = ["Block of flats", "House", "Municipal"]
    occupations = ["Sales staff", "Managers", "Core staff", "None"]

    records = []
    for _ in range(10):
        record = {
            "SK_ID_CURR": random.randint(456255, 999999),
            "TARGET": random.choice([0, 1]),
            "NAME_CONTRACT_TYPE": random.choice(contract_types),
            "CODE_GENDER": random.choice(genders),
            "FLAG_OWN_CAR": random.choice(own_car),
            "CNT_CHILDREN": random.randint(0, 5),
            "AMT_INCOME_TOTAL": round(random.uniform(117000000, 117100000), 2),
            "NAME_FAMILY_STATUS": random.choice(family_status),
            "REGION_POPULATION_RELATIVE": round(random.uniform(0.001, 0.05), 6),
            "FLAG_MOBIL": 1,
            "FLAG_EMP_PHONE": random.choice([0, 1]),
            "CNT_FAM_MEMBERS": random.randint(1, 6),
            "HOUSETYPE_MODE": random.choice(house_types),
            "OCCUPATION_TYPE": random.choice(occupations),
            "AGE": random.randint(20, 70)
        }
        records.append(record)
    
    return pd.DataFrame(records)

In [ ]:
# Function for real-time data fetching
def fetch_data():
    all_data = [] 

    print("\nInitializing real-time credit card data fetch...\n")
    time.sleep(1)
    
    while True:
        user_input = input("Do you want to fetched updated credit card record? (yes to start, stop to end): ").strip().lower()
        
        if user_input == 'yes':
            print("Fetching new credit card record...", end="")
            sys.stdout.flush()
            time.sleep(random.uniform(0.5, 1.5))

            data = fetching_real_time_data()
            print(" Data fetched successfully!")
            all_data.append(data)
        elif user_input == 'stop':
            print("\nStopping the data collection. Finalizing...\n")
            time.sleep(1)
            break
        else:
            print("Invalid input. Please enter 'yes' to generate a record or 'stop' to end.")
    
    if all_data:
        print("\nFinalizing the dataset...\n")
        time.sleep(1)
        print("Merging all generated records...\n")
        print("Inserted new records")
        return pd.concat(all_data, ignore_index=True)
    else:
        print("No records fetched, returning an empty DataFrame.")  
        return pd.DataFrame(columns=['TransactionID', 'Amount'])  

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>3. Load HuggingFace Model</b>
<p style = 'font-size:16px;font-family:Arial;'>To generate embeddings, we need an ONNX model capable of transforming text into vector representations. We use a pretrained model from [Teradata's Hugging Face repository](https://huggingface.co/Teradata/gte-base-en-v1.5), such as gte-base-en-v1.5. The model and its tokenizer are downloaded and stored in Vantage tables as BLOBs using the save_byom function.</p>

In [ ]:
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"

In [ ]:
from huggingface_hub import hf_hub_download

model_name = "bge-small-en-v1.5"
number_dimensions_output = 384
model_file_name = "model.onnx" 

In [ ]:
# Step 1: Download Model from Teradata HuggingFace Page

hf_hub_download(repo_id=f"Teradata/{model_name}", filename=f"onnx/{model_file_name}", local_dir="./")
hf_hub_download(repo_id=f"Teradata/{model_name}", filename=f"tokenizer.json", local_dir="./")

In [ ]:
try:
    db_drop_table("embeddings_models")
except:
    pass
try:
    db_drop_table("embeddings_tokenizers")
except:
    pass

In [ ]:
# Step 2: Load Models into Vantage
# a) Embedding model
save_byom(model_id = model_name, # must be unique in the models table
               model_file = f"onnx/{model_file_name}",
               table_name = 'embeddings_models' )
# b) Tokenizer
save_byom(model_id = model_name, # must be unique in the models table
              model_file = 'tokenizer.json',
              table_name = 'embeddings_tokenizers') 

<p style = 'font-size:16px;font-family:Arial;'>Recheck the installed model and tokenizer

In [ ]:
df_model = DataFrame('embeddings_models')
df_model

In [ ]:
df_token = DataFrame('embeddings_tokenizers')
df_token

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>3. Create the Embeddings</b>

<p style = 'font-size:16px;font-family:Arial;'> Let us take a look at our data.

In [ ]:
DF_CC = DataFrame(in_schema("DEMO_CreditCard","Credit_Card"))
DF_CC

In [ ]:
DF_CC.shape

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>3.2 Creation of Views and Final Embeddings Table
</b>

<p style = 'font-size:16px;font-family:Arial;'>
Now it's time to generate the embeddings using <b>ONNXEmbeddings</b>.<br>We run the ONNXEmbeddings function to generate embeddings for a small subset of records. The model is <b>loaded into the cache memory on each node</b>, and Teradata's <b>Massively Parallel Processing (MPP)</b> architecture ensures that embeddings are computed in parallel using <b>ONNX Runtime</b> on each node.  <br>Having said that, generating embeddings for the entire training set can be time-consuming, especially when working on a system with limited resources. In the <b>ClearScape Analytics experience</b>, only a <b>4 AMP system</b> with constrained RAM and CPU power is available. To ensure smooth execution, we test embedding generation on a small sample and use <b>pre-calculated embeddings</b> for the remainder of demo. In a real-life scenario you would tyipically encounter multiple hundred AMPs with much more compute power!<br>Also have a look at the most important input parameters of this <b>ONNXEmbeddings</b> function.
<ul style = 'font-size:16px;font-family:Arial;'>
<li><b>InputTable</b>: The source table containing the text to be embedded. </li>
<li><b>ModelTable</b>: The table storing the ONNX model.                    </li>
<li><b>TokenizerTable</b>: The table storing the tokenizer JSON file.       </li>
<li><b>Accumulate</b>: Specifies additional columns to retain in the output </li>  
<li><b>OutputFormat</b>: Specifies the data format of the output embeddings (<b>FLOAT32(768)</b>, matching the model's output dimension).</li>
</ul>
<p style = 'font-size:16px;font-family:Arial;'>
Since embedding generation is computationally expensive, we only process <b>10 records for testing</b> and rely on precomputed embeddings for further analysis.  
</p>


<p style='font-size:18px;font-family:Arial;'><b>Generate Embeddings</b></p>    

<p style='font-size:16px;font-family:Arial;'>
Generating embeddings will take approximately <b>10-15 minutes.</b>
</p>


<center><img src="images/visual.svg" alt="embeddings_decision" width=1300 height=1400/></center>


<div class="alert alert-block alert-info">
<p style='font-size:16px;font-family:Arial;'>
<i><b>
These embeddings will later be used in anomaly detection by comparing the similarity between different transactions. By converting each transaction into a vector representation, we can identify outliers or anomalies based on the distance between vectors.</b></i>
</p>
</div>


In [ ]:
configure.byom_install_location = "mldb"

In [ ]:
DF_sample1000 = DataFrame.from_query("SELECT top 1000 SK_ID_CURR AS id, cast(AMT_INCOME_TOTAL as VARCHAR(50)) AS txt FROM credit_card_db t")

In [ ]:
my_model = DataFrame.from_query(f"select * from embeddings_models where model_id = '{model_name}'")
my_tokenizer = DataFrame.from_query(f"select model as tokenizer from embeddings_tokenizers where model_id = '{model_name}'")

In [ ]:
DF_embeddings_sample = ONNXEmbeddings(
    newdata = DF_sample1000,
    modeldata = my_model, 
    tokenizerdata = my_tokenizer, 
    accumulate = ["id", "txt"],
    model_output_tensor = "sentence_embedding",
    output_format = f'FLOAT32({number_dimensions_output})',
    enable_memory_check = False
).result

In [ ]:
DF_embeddings_sample.show_query()

In [ ]:
DF_embeddings_sample.head(2)

In [ ]:
copy_to_sql(DF_embeddings_sample, table_name="DF_embeddings_table" , if_exists='replace')

<hr style="height:1px;border:none;background-;">
<b style = 'font-size:18px;font-family:Arial;'>3.3 Elbow Method
</b>

<p style = 'font-size:16px;font-family:Arial;'>In this step, we apply the <b>Elbow Method</b> to determine the optimal number of clusters for the KMeans clustering algorithm. The Elbow Method helps in selecting the number of clusters that best represents the data by analyzing the <b>Total Within-Cluster Sum of Squares (Total_WithinSS)</b>.</p>

<p style='font-size:16px;font-family:Arial;'>
For each number of clusters, we print the `Total_WithinSS` value, which represents the compactness of the clusters.
</p>


In [ ]:
# Extract embedding features (excluding ID and text columns)
embedding_column_list = [col for col in DF_embeddings_sample.columns if col not in ["id", "txt"]]

total_withinss_values = []

for num_clusters in range(5, 10):
    # Perform KMeans clustering
    kmeans_out = KMeans(data=DataFrame('DF_embeddings_table'), #DF_embeddings_sample,
                        id_column="id",
                        target_columns=embedding_column_list,
                        num_clusters=num_clusters,
                        num_init=10,
                        iter_max=50
                        )
    result_table = kmeans_out.result
    result_table_df = result_table[['td_modelinfo_kmeans']]
    df = result_table_df.assign(has_name = result_table_df.td_modelinfo_kmeans.str.contains('Total_WithinSS', na = False))
    df1 = df[df.has_name == 1]
    df2 = df1.drop(columns =['has_name'])
    df3 = dict(df2.to_pandas()['td_modelinfo_kmeans'])
    
    # Access the value associated with key 0
    value_str = df3[0]

    # Split the string by ':' and strip any leading/trailing whitespace
    numeric_str = value_str.split(':')[1].strip()
    numeric_value = round(float(numeric_str), 2)
    total_withinss_values.append(numeric_value)
    
    # Display the numeric value for each number of clusters
    print(f'Number of clusters: {num_clusters}, Total_WithinSS: {numeric_value}')

<p style='font-size:16px;font-family:Arial;'>
The resulting plot provides a visual representation of the relationship between the number of clusters and the <b>Total_WithinSS</b>. The "elbow" in the plot will suggest the optimal `k` for further analysis.
</p>

In [ ]:
# Plotting the elbow graph
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(range(5, 10), total_withinss_values, marker='o', linestyle='--', color='b')
plt.xlabel('Number of Clusters')
plt.ylabel('Total Within-Cluster Sum of Squares (Total_WithinSS)')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.grid(True)
plt.show()

<p style='font-size:16px;font-family:Arial;'>
In this step, we calculate the differences between successive WCSS values and use the second-order differences to identify the <b>Elbow Point</b>, which helps in selecting the optimal number of clusters (`k`).
</p>

In [ ]:
import numpy as np

# Calculate the differences between successive WCSS values
wcss_diff = np.diff(total_withinss_values)

# Calculate the second-order differences to find the largest drop
wcss_diff2 = np.diff(wcss_diff)

# Find the index of the maximum curvature (Elbow Point)
optimal_k_index = np.argmin(wcss_diff2) + 2 

print(f'Optimal number of clusters (K) based on Elbow Method: {optimal_k_index}')

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>4. Run K-Means on the Embeddings Store and then build final table with Cluster ID assignments to rows</b>

<p style = 'font-size:16px;font-family:Arial;'>The K-means() function groups a set of observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid). This algorithm minimizes the objective function, that is, the total Euclidean distance of all data points from the center of the cluster</p>

In [ ]:
# Convert to Teradata-compatible int
optimal_k_index = int(optimal_k_index)

In [ ]:
embedding_column_list = [col for col in DF_embeddings_sample.columns if col not in ["id", "txt"]]

num_clusters = optimal_k_index
kmeans_out = KMeans(
    id_column="id",
    data=DataFrame('DF_embeddings_table'),
    target_columns=embedding_column_list,
    num_init=10,
    num_clusters=num_clusters,
    iter_max=50,
)

<p style = 'font-size:16px;font-family:Arial;'>The output below shows cluster assignment for each row.</p>

In [ ]:
# Print out the result
kmeans_out.result

<p style = 'font-size:16px;font-family:Arial;'>Let's check how many data points each cluster has.</p>

In [ ]:
# Applying KMeans Predict
KMeansPredict_out_1 = KMeansPredict(data=DataFrame('DF_embeddings_table'),
                                    object=kmeans_out.result,
                                    #accumulate="ram",
                                    output_distance=False)

In [ ]:
# Print the result DataFrames.
KMeansPredict_out_1.result

<p style = 'font-size:16px;font-family:Arial;'>Let's check how many data points each cluster has.</p>

In [ ]:
# Count of each clustered
kmeans_df = KMeansPredict_out_1.result
d2 = kmeans_df.groupby('td_clusterid_kmeans').count()
d2

In [ ]:
# Combine embeddings and clusterid
clustered_df = DataFrame('DF_embeddings_table').join(
    other = kmeans_df,
    on = ["id"],
    how = "inner",
    lprefix="l",
    rprefix="r"
)

clustered_df.shape

In [ ]:
# Storing into database
copy_to_sql(df = clustered_df, table_name = "clustered_data", if_exists="replace")

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>5. Visualization</b>

<hr style='height:1px;border:none;background-;'> 

<p style='font-size:18px;font-family:Arial;'><b>5.1 Visualization of Clusters</b></p> 

<p style='font-size:16px;font-family:Arial;'>The graph illustrates the clustering of transactions into distinct groups for credit card fraud detection using anomaly detection. Based on the analysis, the data has been divided into optimal clusters, each representing a unique transaction pattern. This clustering approach helps identify potential fraudulent activities by distinguishing normal and anomalous transaction behaviors, enabling more targeted fraud detection and prevention efforts.</p>


<p style='font-size:16px;font-family:Arial;'>This visualization helps us explore the structure of the clusters and visually identify <b>anomalies</b> (points that deviate significantly from their respective cluster centers). The use of <b>diamonds for anomalies</b> and <b>circles for normal data points</b> provides a clear and intuitive way to distinguish between outliers and inliers in the data, making it easier to detect potential fraudulent or unusual activity.</p>

<p style='font-size:16px;font-family:Arial;'>The interactive nature of the plot allows for an engaging exploration of the data, where users can hover over points to view more detailed information.</p>


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
# import mplcursors  # for hover tooltips (optional)

clus = clustered_df.to_pandas()
# --- Perform t-SNE ---
tsne = TSNE(n_components=2, random_state=123)
tsne_result = tsne.fit_transform(clus.iloc[:, 3:-1])

# --- Create visualization DataFrame ---
tsne_df = pd.DataFrame(tsne_result, columns=['tsne_1', 'tsne_2'])
tsne_df['cluster_id'] = clus['td_clusterid_kmeans']
tsne_df['customer_id'] = clus['l_id']
tsne_df['amount'] = clus['txt']  # rename for clarity

# Truncate text values
tsne_df['truncated_amount'] = tsne_df['amount'].apply(
    lambda x: f"{x[:50]}..." if isinstance(x, str) else x
)

# --- Compute cluster centers ---
cluster_centers = tsne_df.groupby('cluster_id')[['tsne_1', 'tsne_2']].mean()

# --- Compute distance to cluster center ---
def euclidean_distance(row):
    center = cluster_centers.loc[row['cluster_id']]
    return np.sqrt((row['tsne_1'] - center['tsne_1'])**2 + (row['tsne_2'] - center['tsne_2'])**2)

tsne_df['distance'] = tsne_df.apply(euclidean_distance, axis=1)

# --- Mark anomalies per cluster (top 5% distance) ---
tsne_df['is_anomaly'] = tsne_df.groupby('cluster_id')['distance'].transform(
    lambda x: x > x.quantile(0.95)
)

# --- Plot using Matplotlib ---
plt.figure(figsize=(12, 9))
palette = sns.color_palette('tab10', n_colors=tsne_df['cluster_id'].nunique())

# Draw normal and anomaly points separately
for i, cluster in enumerate(sorted(tsne_df['cluster_id'].unique())):
    cluster_data = tsne_df[tsne_df['cluster_id'] == cluster]
    
    # Normal points
    normal_points = cluster_data[~cluster_data['is_anomaly']]
    plt.scatter(
        normal_points['tsne_1'], normal_points['tsne_2'],
        label=f'Cluster {cluster}',
        color=palette[i],
        alpha=0.7,
        edgecolor='k',
        s=50,
        marker='o'
    )
    
    # Anomalies
    anomalies = cluster_data[cluster_data['is_anomaly']]
    plt.scatter(
        anomalies['tsne_1'], anomalies['tsne_2'],
        color=palette[i],
        marker='D',  # diamond marker
        edgecolor='black',
        s=120,
        label=f'Anomaly (Cluster {cluster})'
    )

# --- Plot cluster centers ---
plt.scatter(
    cluster_centers['tsne_1'], cluster_centers['tsne_2'],
    color='black', s=250, marker='X', label='Cluster Centers'
)

plt.title('t-SNE Visualization of Clusters with Anomaly Detection', fontsize=16, weight='bold')
plt.xlabel('Dimension-1', fontsize=14)
plt.ylabel('Dimension-2', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()


plt.show()

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>6. Real-time data Fetching</b>

<hr style='height:1px;border:none;background-;'> 

<p style='font-size:18px;font-family:Arial;'><b>6.1 Data Fetching Process</b></p> 

<p style='font-size:16px;font-family:Arial;'>The <code>fetch_data()</code> function allows the user to fetch new credit card transaction records in real-time. By entering <b>'yes'</b>, the system fetches and stores 10 new records, simulating live data collection. Entering <b>'stop'</b> terminates the data fetching process, finalizing and merging all collected records into a dataset.</p>





In [ ]:
# Fetch Real time credit card data
tdf = fetch_data()

In [ ]:
# Store in database
copy_to_sql(tdf, table_name="credit_card_db", if_exists="replace")

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>7. Create the Embeddings</b>

<p style='font-size:16px;font-family:Arial;'>we generate and store the embeddings for the credit card transaction data. The embeddings are created using a pre-trained model, which transforms the transaction data into vectorized representations that can be used for anomaly detection.</p>

In [ ]:
tdf = DataFrame.from_table('credit_card_db')
tdf.shape

In [ ]:
DF_sample1000 = DataFrame.from_query("SELECT SK_ID_CURR AS id, cast(AMT_INCOME_TOTAL as VARCHAR(50)) AS txt FROM credit_card_db t")

In [ ]:
# Generate Embeddings
DF_embeddings_sample = ONNXEmbeddings(
    newdata = DF_sample1000,
    modeldata = my_model, 
    tokenizerdata = my_tokenizer, 
    accumulate = ["id", "txt"],
    model_output_tensor = "sentence_embedding",
    output_format = f'FLOAT32({number_dimensions_output})',
    enable_memory_check = False
).result
print("All steps completed successfully!")

In [ ]:
# View sample data
DF_embeddings_sample.sample(2)

In [ ]:
# View shape of dataframe
DF_embeddings_sample.shape

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>8. Applying KMeans Prediction</b>

<p style='font-size:16px;font-family:Arial;'>We apply the <b>KMeans Prediction</b> to predict the cluster assignments for the dataset using the previously trained KMeans model. This allows us to label the data points with the clusters they belong to and further analyze the results.</p>

In [ ]:
# Applying KMeansPredict
KMeansPredict_out_1 = KMeansPredict(data=DF_embeddings_sample,
                                    object=kmeans_out.result,
                                    #accumulate="ram",
                                    output_distance=False)

# Print the result DataFrames.
KMeansPredict_out_1.result

<p style = 'font-size:16px;font-family:Arial;'>Let's check how many data points each cluster has.</p>

In [ ]:
# Count of each clusterid
kmeans_df = KMeansPredict_out_1.result
d2 = kmeans_df.groupby('td_clusterid_kmeans').count()
d2

In [ ]:
# Combine embeddings and clusterid
clustered_df = DF_embeddings_sample.join(
    other = kmeans_df,
    on = ["id"],
    how = "inner",
    lprefix="l",
    rprefix="r"
)

In [ ]:
# Store in database
copy_to_sql(df = clustered_df, table_name = "clustered_data", if_exists="append")

In [ ]:
# View updated data
clustered_df = DataFrame.from_table('clustered_data')
clustered_df.shape

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>5. Visualization</b>

<hr style='height:1px;border:none;background-;'> 

<p style='font-size:18px;font-family:Arial;'><b>5.1 Visualization of Clusters</b></p> 

<p style='font-size:16px;font-family:Arial;'>This visualization helps us explore the structure of the clusters and visually identify <b>anomalies</b> (points that deviate significantly from their respective cluster centers). The use of <b>diamonds for anomalies</b> and <b>circles for normal data points</b> provides a clear and intuitive way to distinguish between outliers and inliers in the data, making it easier to detect potential fraudulent or unusual activity.</p>

<p style='font-size:16px;font-family:Arial;'>The interactive nature of the plot allows for an engaging exploration of the data, where users can hover over points to view more detailed information.</p>


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
# import mplcursors  # for hover tooltips (optional)

clus = clustered_df.to_pandas()
# --- Perform t-SNE ---
tsne = TSNE(n_components=2, random_state=123)
tsne_result = tsne.fit_transform(clus.iloc[:, 3:-1])

# --- Create visualization DataFrame ---
tsne_df = pd.DataFrame(tsne_result, columns=['tsne_1', 'tsne_2'])
tsne_df['cluster_id'] = clus['td_clusterid_kmeans']
tsne_df['customer_id'] = clus['l_id']
tsne_df['amount'] = clus['txt']  # rename for clarity

# Truncate text values
tsne_df['truncated_amount'] = tsne_df['amount'].apply(
    lambda x: f"{x[:50]}..." if isinstance(x, str) else x
)

# --- Compute cluster centers ---
cluster_centers = tsne_df.groupby('cluster_id')[['tsne_1', 'tsne_2']].mean()

# --- Compute distance to cluster center ---
def euclidean_distance(row):
    center = cluster_centers.loc[row['cluster_id']]
    return np.sqrt((row['tsne_1'] - center['tsne_1'])**2 + (row['tsne_2'] - center['tsne_2'])**2)

tsne_df['distance'] = tsne_df.apply(euclidean_distance, axis=1)

# --- Mark anomalies per cluster (top 5% distance) ---
tsne_df['is_anomaly'] = tsne_df.groupby('cluster_id')['distance'].transform(
    lambda x: x > x.quantile(0.95)
)

# --- Plot using Matplotlib ---
plt.figure(figsize=(12, 9))
palette = sns.color_palette('tab10', n_colors=tsne_df['cluster_id'].nunique())

# Draw normal and anomaly points separately
for i, cluster in enumerate(sorted(tsne_df['cluster_id'].unique())):
    cluster_data = tsne_df[tsne_df['cluster_id'] == cluster]
    
    # Normal points
    normal_points = cluster_data[~cluster_data['is_anomaly']]
    plt.scatter(
        normal_points['tsne_1'], normal_points['tsne_2'],
        label=f'Cluster {cluster}',
        color=palette[i],
        alpha=0.7,
        edgecolor='k',
        s=50,
        marker='o'
    )
    
    # Anomalies
    anomalies = cluster_data[cluster_data['is_anomaly']]
    plt.scatter(
        anomalies['tsne_1'], anomalies['tsne_2'],
        color=palette[i],
        marker='D',  # diamond marker
        edgecolor='black',
        s=120,
        label=f'Anomaly (Cluster {cluster})'
    )

# --- Plot cluster centers ---
plt.scatter(
    cluster_centers['tsne_1'], cluster_centers['tsne_2'],
    color='black', s=250, marker='X', label='Cluster Centers'
)

plt.title('t-SNE Visualization of Clusters with Anomaly Detection', fontsize=16, weight='bold')
plt.xlabel('Dimension-1', fontsize=14)
plt.ylabel('Dimension-2', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()


plt.show()

<hr style="height:1px;border:none;background-;">
<p style = 'font-size:20px;font-family:Arial;'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;'>In this demo we have seem that how we can run HuggingFace Embedding Model (BAAI/bge-small-1.5) in ONNX format and run it in database parallelly to create embeddings. We have done KMeans Clustering to group for credit card fraud detection using anomaly detection and interactive t-SNE visualization allowed us to explore the clusters, distinguish anomalies using <b>diamond shapes</b>, and analyze the structure of the data more intuitively.</p> 

<hr style="height:2px;border:none;background-;">
<b style = 'font-size:20px;font-family:Arial;'>6. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['credit_card_embeddings_store', 'credit_card_db', 'clustered_data','DF_embeddings_table']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

views = ['credit_card_tokenized_for_embeddings','credit_card_embeddings']

for view in views:
    try:
        db_drop_view(view_name=view)
    except:
        pass

<hr style="height:1px;border:none;background-;">
<p style = 'font-size:18px;font-family:Arial;'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_CreditCard_cloud');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>

- `SK_ID_CURR`: Customer ID
- `TARGET`: Target (0 = No, 1 = Yes)
- `NAME_CONTRACT_TYPE`: Contract type (e.g., Cash loans, Revolving loans)
- `CODE_GENDER`: Gender (Female / Male)
- `FLAG_OWN_CAR`: Car ownership status (Y = Yes, N = No)
- `CNT_CHILDREN`: Number of children
- `AMT_INCOME_TOTAL`: Total income
- `NAME_FAMILY_STATUS`: Family status (e.g., Married, Separated, Single)
- `REGION_POPULATION_RELATIVE`: Relative population of the region
- `FLAG_MOBIL`: Mobile phone status (1 = Yes)
- `FLAG_EMP_PHONE`: Employment phone status (0 = No, 1 = Yes)
- `CNT_FAM_MEMBERS`: Number of family members
- `HOUSETYPE_MODE`: Type of house (e.g., Block of flats, House, Municipal)
- `OCCUPATION_TYPE`: Occupation (e.g., None, Sales staff, Managers)
- `AGE`: Age of the customer

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>KMeans reference: <a href = 'https://docs.teradata.com/search/all?query=KMeans&value-filters=vrm_release~%252220.00.00.03%2522&content-lang=en-US'>here</a></li>
    <li>KMeansPredict reference: <a href = 'https://docs.teradata.com/search/all?query=KMeansPredict&value-filters=vrm_release~%252220.00.00.03%2522&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>